In [ ]:
username_cols = {2018: 17, 2019: 18}
protected_cols = {'Gender': {2018: 18, 2019: 20}, 
                  'Prior_exp': {2018: 24, 2019: 25}}
pretest_filenames = {2018: "Datasets/2018_REFLECT_Pretest.csv",
                     2019: "Datasets/2019_REFLECT_Pretest.csv"}

In [1]:
from abroca import *
import pandas as pd
import csv
import numpy as np

from copy import deepcopy

import tensorflow
tensorflow.random.set_seed(2)
from tensorflow.keras.models import load_model

from sklearn.metrics import roc_auc_score, RocCurveDisplay, precision_recall_fscore_support, accuracy_score
from statistics import mean, stdev
import matplotlib.pyplot as plt

import import_ipynb
from build_models import *
from evaluation import *
from bias_mitigation import bias_postprocessing

from scipy.stats import pearsonr
from scipy import stats
import re

/home/agupta44/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


importing Jupyter notebook from build_models.ipynb
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from fetch_data.ipynb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Graph was finalized.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from bias_mitigation.ipynb


In [ ]:
def reassign_gender(data):
    return

In [ ]:
def reassign_exp(data):
    for s_id in data:
        if data[s_id]['Prior_exp'] == '2':
            data[s_id]['Prior_exp'] == '1'
        elif data[s_id]['Prior_exp'] == '3' or data[s_id]['Prior_exp'] == '4' or data[s_id]['Prior_exp'] == '5':
            data[s_id]['Prior_exp'] = '2'
    
    return

In [ ]:
def get_protected_attr(data, protected_label="Gender"):
    studies = [2018, 2019]
    for s_id in data:
        found = False
        for year in studies:
            with open(pretest_filenames[year]) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                for row in csv_reader:
                    if row[username_cols[year]] == s_id or row[username_cols[year]] in s_id:
                        found = True
                        data[s_id][protected_label] = row[protected_cols[protected_label][year]]
            if found == True:
                break
                
    if protected_label == "Gender":
        reassign_gender(data)
    elif protected_label == "Prior_exp":
        reassign_exp(data)
                
    return data

In [ ]:
def get_protected_attr_by_id(ids, protected_label="Gender"):
    studies = [2018, 2019]
    attrs = {}
    for s_id in ids:
        found = False
        for year in studies:
            with open(pretest_filenames[year]) as csv_file:
                csv_reader = csv.reader(csv_file, delimiter=',')
                for row in csv_reader:
                    if row[username_cols[year]] == s_id or row[username_cols[year]] in s_id:
                        found = True
                        attrs[s_id] = {'Gender': '-1', 'Prior_exp': '-1'}
                        attrs[s_id][protected_label] = row[protected_cols[protected_label][year]]
            if found == True:
                break
                
    if protected_label == "Gender":
        reassign_gender(attrs)
    elif protected_label == "Prior_exp":
        reassign_exp(attrs)
                
    return attrs

In [ ]:
def create_dataframe(data, protected_label="Gender", label_name="posttest"):
    new_dict = {'ids': [], protected_label: [], 'y_pred': [], 'y_true': []}
    for s_id in data:
        count = len(data[s_id]['y_pred'])
        #new_dict[protected_label].extend([data[s_id][protected_label] for i in range(count)])
        new_dict[protected_label].append(data[s_id][protected_label])
        #new_dict['ids'].extend([s_id for i in range(count)])
        new_dict['ids'].append(s_id)
        if label_name == "posttest":
            #new_dict['y_pred'].extend([y for y in data[s_id]['y_pred']])
            #new_dict['y_true'].extend([y>=0.43 for y in data[s_id]['y_true']])
            new_dict['y_pred'].append(mean(data[s_id]['y_pred']))
            m = 0
            if data[s_id]['y_true'][0] > 6/17:
                m = 1
            new_dict['y_true'].append(m)
        elif label_name == "ratings":
            #new_dict['y_pred'].extend([y for y in data[s_id]['y_pred']])
            #new_dict['y_true'].extend([y>=0.5 for y in data[s_id]['y_true']])
            new_dict['y_pred'].append(mean(data[s_id]['y_pred']))
            m = 0
            if data[s_id]['y_true'][0] >= 2.5/5:
                m = 1
            new_dict['y_true'].append(m)
            
    print(len(new_dict['y_pred']), len(new_dict['y_true']))
        
    return pd.DataFrame(data=new_dict)

In [ ]:
def compute_auc(data, protected_label='Gender', value=1, label_name='posttest', disp_ax=None):
    y_pred = []
    y_true = []
    instance_count = 0
    for s_id in data:
        if data[s_id][protected_label] == str(value):
            instance_count += 1
            if label_name == "posttest":
                #y_pred.extend([y for y in data[s_id]['y_pred']])
                #y_true.extend([y>=0.43 for y in data[s_id]['y_true']])
                y_pred.append(mean(data[s_id]['y_pred']))
                m = 0
                if data[s_id]['y_true'][0] > 6/17:
                    m = 1
                y_true.append(m)
            elif label_name == "ratings":
                #y_pred.extend([y for y in data[s_id]['y_pred']])
                #y_true.extend([y>=0.5 for y in data[s_id]['y_true']])
                y_pred.append(mean(data[s_id]['y_pred']))
                m = 0
                if data[s_id]['y_true'][0] >= 2.5/5:
                    m = 1
                y_true.append(m)
      
    print('Instance count: ', instance_count)
    print(protected_label, "[Value: ", value, " AUC: ", roc_auc_score(y_true, y_pred), "]")
    
    legend = {'Gender': {1: 'Male ROC', 2: 'Female ROC'}, 
              'Prior_exp': {1: '0-5hrs ROC', 2: '>5hrs ROC'}}
    name = legend[protected_label][value]
    
    if disp_ax != None:
        return RocCurveDisplay.from_predictions(y_true, y_pred, name=name, ax=disp_ax)
    
    return RocCurveDisplay.from_predictions(y_true, y_pred, name=name)

In [ ]:
def detect_bias(data, label_name="posttest", protected_label="Gender"):
    data = get_protected_attr(data, protected_label=protected_label) 
    df = create_dataframe(data, protected_label=protected_label, label_name=label_name)
    
    #abroca_results = compute_abroca(df, pred_col='y_pred', label_col='y_true',
    #                                protected_attr_col=protected_label, majority_protected_attr_val=1,
    #                                #compare_type='multiple', 
    #                                plot_slices=True)
    #print("ABROCA results for ", label_name, " [", protected_label, "]: \t", abroca_results)
    
    if protected_label == 'Gender':
        disp1 = compute_auc(data, protected_label='Gender', value=1, label_name=label_name)
        disp2 = compute_auc(data, protected_label='Gender', value=2, label_name=label_name,
                            disp_ax=disp1.ax_)
        #compute_auc(data, protected_label='Gender', value=3, label_name=label_name)
    elif protected_label == 'Prior_exp':
        disp1 = compute_auc(data, protected_label='Prior_exp', value=1, label_name=label_name)
        compute_auc(data, protected_label='Prior_exp', value=2, label_name=label_name, 
                    disp_ax=disp1.ax_)
    
    return

In [ ]:
def get_model_bias(model_filename, test_data, get_required_model, label_name="posttest", protected_label="Gender",
                   postprocess=False, algorithm='CalibratedEqOddsPostprocessing'):
    model_trained = load_model(model_filename)
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    y_pred = model_trained.predict(data['Test'])
    y_true = labels['Test']
    
    y_pred = [[label1[0] for label1 in y_pred[0]], [label2[0] for label2 in y_pred[1]]]
    
    if postprocess:
        y_pred[1] = bias_postprocessing(data['Test'], y_true, y_pred[1], ids['Test'], algorithm=algorithm)
        
    posttest_dict = create_dict(y_pred[0], y_true[0], ids['Test']['ids']) 
    #ratings_dict1 = create_dict(y_pred[1], y_true[1], ids['Test']['ids'])
    ratings_dict = create_dict(y_pred[1], y_true[1], ids['Test']['response_ids'])
    
    get_accuracy(posttest_dict, threshold=5/17, label="Posttest")
    get_accuracy(ratings_dict, threshold=2.5/5, label="Ratings")
    
    print("ABROCA results for overall set")
    if label_name == 'posttest':
        detect_bias(posttest_dict, label_name=label_name, protected_label=protected_label) 
    elif label_name == 'ratings':
        detect_bias(ratings_dict, label_name=label_name, protected_label=protected_label) 
    
    result_metrics = {'posttest_macro': get_macro_scores(posttest_dict), 
                      'rating_macro': get_macro_scores(ratings_dict),
                      'posttest_micro': get_micro_scores(y_pred[0], y_true[0]), 
                      'ratings_micro': get_micro_scores(y_pred[1], y_true[1])}
    
    print("OVERALL SET: ", result_metrics)
    
    return

In [ ]:
def get_gametrace_stats(test_data, get_required_model, protected_label="Gender"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': [[] for i in range(8)], '2': [[] for i in range(8)]}
    
    for i,s_id in enumerate(ids):
        gender = genders[s_id][protected_label]
        if gender in gender_logs:
            for idx in range(8):
                gender_logs[gender][idx].append(logs[i][idx])
                
    for gender in gender_logs:
        #gender_logs[gender] = [n/len(ids) for n in gender_logs[gender]]
        print(gender)
        stdevs = []
        means = []
        for idx in range(8):
            #print(idx, stdev(gender_logs[gender][idx]))
            #print(idx, mean(gender_logs[gender][idx]))
            stdevs.append(stdev(gender_logs[gender][idx])*stdev(gender_logs[gender][idx]))
            means.append(stdev(gender_logs[gender][idx])*stdev(gender_logs[gender][idx]))
        print("mean: ", mean(means), " stdev: ", stdev(stdevs))
        
    gender_logs = {'1': [], '2': []}
    
    for i,s_id in enumerate(ids):
        gender = genders[s_id][protected_label]
        if gender in gender_logs:
            gender_logs[gender].extend(logs[i])
            
    for gender in gender_logs:
        print(gender)
        print("Mean: ", mean(gender_logs[gender]))
        print("Stdev: ", stdev(gender_logs[gender]))
    
    return #gender_logs

In [ ]:
def get_gender_r_stats(test_data, get_required_model, protected_label="Gender"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Readings': []}, '2': {'Ratings': [], 'Readings': []}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = 0
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = []
            data_sid[s_id]['Readings'] = []
            data_sid[s_id]['Gender'] = gender
        if response_ids[i] != s_id + str(r_id):
            print(i, response_ids[i])
            reflect_length, response = get_reflection_length(s_id, r_id)
            if reflect_length != 0:
                data_sid[s_id]['Ratings'].append(labels['Train'][1][i-1])
                data_sid[s_id]['Readings'].append(reflect_length)
            r_id += 1
            
    for s_id in data_sid:
        if data_sid[s_id]['Gender'] in gender_logs:
            gender_logs[data_sid[s_id]['Gender']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Gender']]['Readings'].extend(data_sid[s_id]['Readings'])
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings'])
    
    return gender_logs

In [ ]:
def get_gender_gametrace_stats(test_data, get_required_model, protected_label="Gender"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Readings': []}, '2': {'Ratings': [], 'Readings': []}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = 0
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = []
            data_sid[s_id]['Readings'] = []
            data_sid[s_id]['Gender'] = gender
        if response_ids[i] != s_id + str(r_id):
            print(i, response_ids[i])
            data_sid[s_id]['Ratings'].append(labels['Train'][1][i-1])
            data_sid[s_id]['Readings'].append(logs[i-1][1] + logs[i-1][5])
            r_id += 1
            
    for s_id in data_sid:
        if data_sid[s_id]['Gender'] in gender_logs:
            gender_logs[data_sid[s_id]['Gender']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Gender']]['Readings'].extend(data_sid[s_id]['Readings'])
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings'])
    
    return gender_logs

In [ ]:
def get_pretest_reflect_stats(test_data, get_required_model, protected_label="Prior_exp"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Readings': [], 'low': 0, 'high': 0}, 
                   '2': {'Ratings': [], 'Readings': [], 'low': 0, 'high': 0}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = 0
            rating = 0
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = [(labels['Train'][0][i]-data['Train'][2][i])/(1-data['Train'][2][i])]
            data_sid[s_id]['Ratings'] = [data['Train'][2][i]]
            data_sid[s_id]['Readings'] = []
            data_sid[s_id]['Prior_exp'] = gender
        rating, response = get_reflection_length(s_id, int(response_ids[i][len(response_ids[i])-1]))
        if rating != 0:
            data_sid[s_id]['Readings'].append(rating)
            
    for s_id in data_sid:
        try:
            data_sid[s_id]['Readings'] = [mean(data_sid[s_id]['Readings'])]
        except:
            data_sid[s_id]['Readings'] = [0]
        data_sid[s_id]['Prior_exp'] = '1'
        if data_sid[s_id]['Prior_exp'] in gender_logs:
            gender_logs[data_sid[s_id]['Prior_exp']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Prior_exp']]['Readings'].extend(data_sid[s_id]['Readings'])
            if mean(data_sid[s_id]['Readings']) >= 0.5:
                gender_logs[data_sid[s_id]['Prior_exp']]['high'] += 1
            else:
                gender_logs[data_sid[s_id]['Prior_exp']]['low'] += 1
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings'])
        gender_logs[gender]['mean'] = mean(gender_logs[gender]['Readings'])
        gender_logs[gender]['variance'] = stdev(gender_logs[gender]['Readings'])
        print(gender_logs[gender]['p'], gender_logs[gender]['mean'], gender_logs[gender]['variance'],
              gender_logs[gender]['low'], gender_logs[gender]['high'])
        print(stats.ttest_ind(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings']))
    
    return gender_logs

In [ ]:
def get_exp_reflect_stats(test_data, get_required_model, protected_label="Prior_exp"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Readings': [], 'low': 0, 'high': 0}, 
                   '2': {'Ratings': [], 'Readings': [], 'low': 0, 'high': 0}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = 0
            rating = 0
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = [(labels['Train'][0][i]-data['Train'][2][i])/(1-data['Train'][2][i])]
            data_sid[s_id]['Ratings'] = [labels['Train'][0][i]]
            data_sid[s_id]['Readings'] = []
            data_sid[s_id]['Prior_exp'] = gender
        rating, response = get_reflection_length(s_id, int(response_ids[i][len(response_ids[i])-1]))
        if rating != 0:
            data_sid[s_id]['Readings'].append(rating)
            
    for s_id in data_sid:
        try:
            data_sid[s_id]['Readings'] = [mean(data_sid[s_id]['Readings'])]
        except:
            data_sid[s_id]['Readings'] = [0]
        if data_sid[s_id]['Prior_exp'] in gender_logs:
            gender_logs[data_sid[s_id]['Prior_exp']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Prior_exp']]['Readings'].extend(data_sid[s_id]['Readings'])
            if mean(data_sid[s_id]['Readings']) >= 0.5:
                gender_logs[data_sid[s_id]['Prior_exp']]['high'] += 1
            else:
                gender_logs[data_sid[s_id]['Prior_exp']]['low'] += 1
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings'])
        gender_logs[gender]['mean'] = mean(gender_logs[gender]['Readings'])
        gender_logs[gender]['variance'] = stdev(gender_logs[gender]['Readings'])
        print(gender_logs[gender]['p'], gender_logs[gender]['mean'], gender_logs[gender]['variance'],
              gender_logs[gender]['low'], gender_logs[gender]['high'])
    
    return gender_logs

In [ ]:
def get_exp_reflection_stats(test_data, get_required_model, protected_label="Prior_exp"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    response_embs = [n[19] for n in data['Train'][1]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Reflect_length': [], 'low': 0, 'high': 0}, 
                   '2': {'Ratings': [], 'Reflect_length': [], 'low': 0, 'high': 0}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = int(response_ids[i][len(response_ids[i])-1])
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = []
            data_sid[s_id]['Reflect_length'] = []
            data_sid[s_id]['Prior_exp'] = gender
        if response_ids[i] != s_id + str(r_id):
            print(i, response_ids[i])
            reflect_length, response = get_reflection_length(s_id, r_id)
            if reflect_length != 0:
                if reflect_length != 1:
                    data_sid[s_id]['Ratings'].append((labels['Train'][1][i-1]-reflect_length)/(1-reflect_length))
                else:
                    data_sid[s_id]['Ratings'].append((labels['Train'][1][i-1]-reflect_length)/reflect_length)
                data_sid[s_id]['Reflect_length'].append(reflect_length)
            r_id += 1
            
    for s_id in data_sid:
        if data_sid[s_id]['Prior_exp'] in gender_logs:
            gender_logs[data_sid[s_id]['Prior_exp']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Prior_exp']]['Reflect_length'].extend(data_sid[s_id]['Reflect_length'])
            try:
                if mean(data_sid[s_id]['Reflect_length']) >= 0.5:
                    gender_logs[data_sid[s_id]['Prior_exp']]['high'] += 1
                else:
                    gender_logs[data_sid[s_id]['Prior_exp']]['low'] += 1
            except:
                pass
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Reflect_length'], gender_logs[gender]['Ratings'])
        gender_logs[gender]['mean'] = mean(gender_logs[gender]['Reflect_length'])
        gender_logs[gender]['variance'] = stdev(gender_logs[gender]['Reflect_length'])
        print(gender_logs[gender]['p'], gender_logs[gender]['mean'], gender_logs[gender]['variance'],
              gender_logs[gender]['low'], gender_logs[gender]['high'])
    
    return gender_logs

In [ ]:
def get_exp_gametrace_stats(test_data, get_required_model, protected_label="Prior_exp"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Readings': []}, '2': {'Ratings': [], 'Readings': []}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = 0
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = [labels['Train'][0][i]]
            data_sid[s_id]['Readings'] = []
            data_sid[s_id]['Prior_exp'] = gender
        data_sid[s_id]['Readings'].append(logs[i-1][1] + logs[i-1][5])
        r_id += 1
            
    for s_id in data_sid:
        data_sid[s_id]['Readings'] = [mean(data_sid[s_id]['Readings'])]
        if data_sid[s_id]['Prior_exp'] in gender_logs:
            gender_logs[data_sid[s_id]['Prior_exp']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Prior_exp']]['Readings'].extend(data_sid[s_id]['Readings'])
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings'])
    
    return gender_logs

In [ ]:
def get_gender_reflection_stats(test_data, get_required_model, protected_label="Gender"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    response_embs = [n[19] for n in data['Train'][1]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Reflect_length': [], 'Reflections': []}, 
                   '2': {'Ratings': [], 'Reflect_length': [], 'Reflections': []}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = int(response_ids[i][len(response_ids[i])-1])
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = []
            data_sid[s_id]['Reflect_length'] = []
            data_sid[s_id]['Gender'] = gender
        if response_ids[i] != s_id + str(r_id):
            print(i, response_ids[i])
            reflect_length = get_reflection_length(s_id, r_id)
            if reflect_length != 0:
                data_sid[s_id]['Ratings'].append(labels['Train'][1][i-1])
                data_sid[s_id]['Reflect_length'].append(reflect_length)
            r_id += 1
            
    for s_id in data_sid:
        if data_sid[s_id]['Gender'] in gender_logs:
            gender_logs[data_sid[s_id]['Gender']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Gender']]['Reflect_length'].extend(data_sid[s_id]['Reflect_length'])
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Reflect_length'], gender_logs[gender]['Ratings'])
    
    return gender_logs

In [ ]:
def get_reflection_length(s_id, r_id):
    with open('Datasets/LabeledReflections.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if row[0] == s_id and row[1] == str(r_id):
                return len(re.findall(r'\w+', row[5])), row[5]
                #eturn float(row[8])/5, row[5]
    
    print("ALERT")
    return 0, ""

In [ ]:
def get_gender_reflection_stats(test_data, get_required_model, protected_label="Gender"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    response_embs = [n[19] for n in data['Train'][1]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Reflect_length': [], 'Reflections': []}, 
                   '2': {'Ratings': [], 'Reflect_length': [], 'Reflections': []}}
    data_sid = {}
    
    for i, s_id in enumerate(ids):
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = int(response_ids[i][len(response_ids[i])-1])
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = []
            data_sid[s_id]['Reflect_length'] = []
            data_sid[s_id]['Reflections'] = []
            data_sid[s_id]['Gender'] = gender
        if response_ids[i] != s_id + str(r_id):
            print(i, response_ids[i])
            reflect_length, response = get_reflection_length(s_id, r_id)
            if reflect_length != 0:
                data_sid[s_id]['Ratings'].append(labels['Train'][0][i-1])
                data_sid[s_id]['Reflect_length'].append(reflect_length)
                data_sid[s_id]['Reflections'].append(response)
            r_id += 1
            
    for s_id in data_sid:
        if data_sid[s_id]['Gender'] in gender_logs:
            gender_logs[data_sid[s_id]['Gender']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Gender']]['Reflect_length'].extend(data_sid[s_id]['Reflect_length'])
            gender_logs[data_sid[s_id]['Gender']]['Reflections'].extend(data_sid[s_id]['Reflections'])
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Reflect_length'], gender_logs[gender]['Ratings'])
        bin_ratings = [r>2.5 for r in gender_logs[gender]['Ratings']]
    
    return gender_logs

In [ ]:
def get_accuracy(data, threshold=0.5, label="Posttest"):
    pred_labels = []
    true_labels = []
    
    for s_id in data:
        pred_labels.append(mean(data[s_id]['y_pred']) >= threshold)
        true_labels.append(mean(data[s_id]['y_true']) >= threshold)
            
    print(label, "accuracy: ", accuracy_score(true_labels, pred_labels))
    print(precision_recall_fscore_support(true_labels, pred_labels, average=None))
    print(precision_recall_fscore_support(true_labels, pred_labels, average='micro'))
    print("AUC: ", roc_auc_score(true_labels, pred_labels))
    
    return

In [ ]:
def get_exp_game_stats(test_data, get_required_model, protected_label="Prior_exp"):
    raw_data = {'Train': deepcopy(test_data),
                'Val': deepcopy(test_data),
                'Test': deepcopy(test_data)}
    model, data, labels, ids = get_required_model(raw_data, seq_length=20)
    logs = [n[19] for n in data['Train'][0]]
    print("got logs")
    response_ids = ids['Train']['response_ids']
    ids = ids['Train']['ids']
    genders = get_protected_attr_by_id(ids, protected_label=protected_label)
    gender_logs = {'1': {'Ratings': [], 'Readings': []}, '2': {'Ratings': [], 'Readings': []}}
    data_sid = {}
    durations = get_durations()
    
    for i, s_id in enumerate(durations):
        if s_id not in ids:
            continue
        if s_id not in data_sid:
            data_sid[s_id] = {}
            r_id = 0
            gender = genders[s_id][protected_label]
            data_sid[s_id]['Ratings'] = [labels['Train'][0][ids.index(s_id)]]
            data_sid[s_id]['Readings'] = []
            data_sid[s_id]['Prior_exp'] = gender
            data_sid[s_id]['Readings'] = durations[s_id]['reading_durations']
        r_id += 1
            
    for s_id in data_sid:
        data_sid[s_id]['Readings'] = [mean(data_sid[s_id]['Readings'])]
        if data_sid[s_id]['Prior_exp'] in gender_logs:
            gender_logs[data_sid[s_id]['Prior_exp']]['Ratings'].extend(data_sid[s_id]['Ratings'])
            gender_logs[data_sid[s_id]['Prior_exp']]['Readings'].extend(data_sid[s_id]['Readings'])
        
    for gender in gender_logs:
        gender_logs[gender]['p'] = pearsonr(gender_logs[gender]['Readings'], gender_logs[gender]['Ratings'])
    
    return gender_logs

In [ ]:
def get_durations():
    data = {}
    event_files = ['Datasets/EventSequence2018.csv', 'Datasets/EventSequence2019.csv']
    
    for filename in event_files:
        with open(filename) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            row_ct = 0
            for row in csv_reader:
                if row_ct == 0:
                    row_ct += 1
                    continue
                if row[0] not in data:
                    data[row[0]] = {'reading_durations': [], 'cumulative_duration': 0}
                if row[1] == 'BooksAndArticles' or row[1] == 'Posters':# or row[1] == 'Conversation': 
                    try:
                        data[row[0]]['cumulative_duration'] += float(row[3])
                    except:
                        pass
                    data[row[0]]['reading_durations'].append(data[row[0]]['cumulative_duration'])
                    
                    
    for s_id in data:
        data[s_id]['mean_duration'] = sum(data[s_id]['reading_durations'])
        data[s_id]['stdev_duration'] = stdev(data[s_id]['reading_durations'])
        
    return data

In [ ]:
def get_reflection_attempts(data):
    counts = []
    for s_id in data:
        try:
            counts.append(data[s_id]['logs'][len(data[s_id]['logs'])-1]['response_number'] + 1)
        except:
            pass
            
    print(mean(counts), stdev(counts))